(cont:mm)=
# Memory Management

- Introduction: Fundamental jobs of the memory manager:
    - Abstracting a useful memory model.
    - Managing the physical memory on the system.
    - Providing and managing a private virtual address space for each user.

- The memory manager is responsible for:
    - 1.) Managing all of the RAM or physical memory on the whole system. 
    - 2.) Creating and managing virtual memory for each process. 
    - 3.) Transitioning the processor(s) between physical memory mode and virtual memory mode as the kernel boots and initializes.

(cont:mm:overview)=
## Overview of MM
- Simple physical memory management.
    - Sharing one physical address space.
    - Relocating programs.
- Virtual memory management: talk about all the advanges 
    - Segmentation.
        - Single segments per address space
        - Multiple segments per address space.
        - Swapping.
        - Compaction.
    - Paging: - now it could be unlimited... 
        - Virtual pages and page frames.
        - Virtual address spaces larger than RAM.
        - Page tables brief (more later)
    - Quick overview of what memory manager needs to do
        - compaction 
        - bringing things in, demand paging
            swapping, paging
        - deciding what to kick out - reclamation
        - anonymous memory
        - file backed and caching storage
        - example: look at elf, parts of the program...

(cont:mm:reclaim)=
## Memory reclaiming algorithms.
- Global page reclaiming
   - FIFO, LRU, NRU...
- Local page reclaiming
   - Reclaiming within a specific address space
- Working set page reclaiming

(cont:mm:ds)=
## Data structures
We need to keep track of ... what is free, what is used... 
- Free lists
   - coalescing free memory.
- In-use page lists
   - active and inactive lists for reclaiming.
   - Buffer Cache 
       - Using RAM to cache file system data & metadata
           - Global page hash
           - Per file page hash

(cont:mm:pf)=
## Page Faults
- File backed page faults
- Anonymous page faults
- Copy On Write page faults

(cont:mm:HW)=
## Memory management hardware
   - Translations
       - Page Table entries
       - Page tables
           - Multiple level page tables
           - Page table entries
           - Inverted page tables
           - Variable page sizes
   - TLB - Transpation Lookaside Buffers and entries
       - Global TLB entries
       - Local TLB entries
       - Tagging TLB entries
       

(cont:mm:dynamics)=
## Paging Dynamics
- Boot-time
- Consuming physical memory
   - Kernel page wiring
   - Page faults
   - Buffer Cache
- Freeing physical memory
   - Process exiting
   - un-mapping
   - Page reclaiming
       - Paging
       - Swapping
       - Page cache purging
- System shutdown

(cont:mm:linux)=
## Memory management in the real world

Linux specific memory management - examples from a real system


Memory Management Introduction

The memory management subsystem has 3 basic responsibilities; 
1. Providing an abstraction so that every user appears to have total and unique access to all of the memory on the system.  
2. Managing all of the physical memory on the system.  
3. Providing and managing a unique private virtual address space for every process currently running on the system. 

Very basic systems do not abstract memory at all.  Instead, programs are loaded directly into physical memory
and all the addresses presented to those programs are physical addresses.  This means that programs running in
a physical memory model must be relocatable because its not known what address they will be loaded and run at.
Being relocatable means that all addresses are either relative or they can be fixed up whenever the program is loaded and/or
moved to a different physical address.  

No memory abstraction also implies that there is no protection between multiple programs loaded into memory at the same time.
If a program takes a wild branch or writes to a random address it will corrupt other programs or even the entire system.

## Notes

Refer [here](https://jappavoo.github.io/UndertheCovers/textbook/unix/shellintro.html#arithmetic-expansion) for doing binary math in shell 